crossref docs: https://github.com/CrossRef/rest-api-doc#readme
crossref py: https://github.com/fabiobatalha/crossrefapi
crossref R: https://github.com/ropensci/rcrossref


APIs used list: https://www.crossref.org/  

Possible: importing data into zotero?

In [2]:
pip install crossrefapi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.1/143.1 kB 3.6 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.0.7
    Uninstalling urllib3-2.0.7:
      Successfully uninstalled urllib3-2.0.7


In [3]:
from urllib.request import urlopen

In [4]:
from crossref.restful import Works

In [5]:
import json

In [6]:
import pandas as pd

In [7]:
from tqdm import tqdm

In [133]:
class return_articles():
  def __init__(self, query_term, year):
    self.query_term = query_term
    self.year = year

  def generate_output_list(self):
    print("--- Generating Unfiltered Output ------------------")
    output_list = []
    works = Works()
    start_date = self.year + "-01-01"
    end_date = self.year + "-12-31"
    query = works.query(self.query_term).filter(type = "journal-article", from_online_pub_date= start_date, until_online_pub_date = end_date)
    print("Total number of query results: " + str(query.count()))
    for item in tqdm(query):
      output_list.append(item)
    return output_list

  def generate_output_titles(self, input_list):
    print("--- Generating Filtered Output --------------------")
    output_titles = []
    output_json = []
    for i in input_list:
      if "title" in i.keys() and self.query_term in i["title"][0].lower():
        output_titles.append(i["title"][0])
        output_json.append(i)

    return output_json

  def generate_output_df(self, filtered_json, required_elements):
    print("--- Generating Dataframe --------------------------")
    dict_list = []
    for i in range(0, len(filtered_json)):
      edited_json = dict.fromkeys(required_elements)
      for j in required_elements:
        edited_json[j] = filtered_json[i][j]

      dict_list.append(edited_json)
    df = pd.DataFrame.from_records(dict_list)

    df.to_csv(self.query_term + " " + self.year + " df.csv", index = False)

    return df


In [138]:
years = ["2022", "2021", "2020"]

In [ ]:
for i in years:
  print("Generating for: " + i)
  articles_gen = return_articles("access to data", i)
  unfiltered_json = articles_gen.generate_output_list()
  filtered_json = articles_gen.generate_output_titles(unfiltered_json)
  print(str(len(filtered_json)) + " filtered results in the year: " + i)
  output_df = articles_gen.generate_output_df(filtered_json, ["title", "DOI", "publisher"])

Generating for: 2022
--- Generating Unfiltered Output ------------------
Total number of query results: 66612


42001it [11:22, 43.09it/s]

In [122]:
works = Works()
query = works.query("access to data").filter(type = "journal-article", from_online_pub_date= "2023-01-01", until_online_pub_date = "2023-12-31").url

In [123]:
query

'https://api.crossref.org/works?filter=type%3Ajournal-article%2Cfrom-online-pub-date%3A2023-01-01%2Cuntil-online-pub-date%3A2023-12-31&query=access+to+data'